In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:15 http://archive.ubuntu.com/ubuntu bionic-backports 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-26 22:17:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  5.64MB/s    in 0.2s    

2022-04-26 22:17:32 (5.64 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)


### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import sum,avg,max,min,mean,count

# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [81]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [82]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).dropDuplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|011040047X|Purple Hard Case ...|
|0110400550|Pink & White 3d M...|
|0133070565|Samsung Galaxy S ...|
|0214614700|Cool Summer Breez...|
|0214714705|Cool Summer Breez...|
|0471295418|New Trading Dimen...|
|0594011868|Nook Simple Touch...|
|0594017580|Jonathan Adler Pu...|
|0594033896|Nook Color Clear ...|
|0594033926|Barnes & Noble De...|
|0594450705|Nook Vara Stylus ...|
|0594459451|Barnes & Noble No...|
|059446790X|2-way Cover Stand...|
|0594467926|Barnes and Noble ...|
|0594479525|Barnes & Noble Va...|
|0594482569|Nook HD+ 9 Inch P...|
|059448426X|Composition Cover...|
|0594513189|NOOK HD 7" Cover ...|
|0594513294|NOOK HD Austen Qu...|
|0594513324|NOOK HD 7" Cover ...|
+----------+--------------------+
only showing top 20 rows



In [83]:
# Create the review_id_table DataFrame.
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702| 2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876| 2015-08-31|
|R3MRWNNR8CBTB7|    1972249|B00U4NATNQ|     577878727| 2015-08-31|
|R1DS6DKTUXAQK3|   10956619|B00SZEFDH8|     654620704| 2015-08

In [84]:
# Create the vine_table. DataFrame
vine_df1 = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df = vine_df1.withColumn("star_rating",
                              vine_df1["star_rating"].cast('int'))
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql:<connectionstring>:5432/postgres"
config = {"user":"", 
          "password":"", 
          "driver":"org.postgresql.Driver"}

In [76]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [58]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [19]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [45]:
# Write vine_df to table in RDS
# 32+ minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)